<a href="https://colab.research.google.com/github/alekz7/MachineLearning/blob/master/titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import numpy as np
import pandas as pd

data = pd.read_csv('./train.csv')
print(data.shape)
# (891, 12)

(891, 12)


In [25]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [26]:
data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [27]:
# La columna edad solo tiene 714 valores asi que llenaremos los faltantes con la media
# actualizacion para llenarlo con la media de los datos combinados
print(data['Age'].median()) # 28
#data['Age'] = data['Age'].fillna(data['Age'].median())

28.0


In [0]:
#TODO graficas con las relaciones de los datos para analizar y descartar
# sexo femenino estaba mas relacionado a sobrevivir
# edad es una caracteristica fuerte, 18 40 parece la edad ideal para sobrevivir
# fare tambien es importante, boletos de alto precio tenian mas oportunidad de sobrevivir
# Embarcacion tambien importa y es una caracteristica fuerte

In [0]:
# vamos a combinar el train y el test para hacer Feature Engineering
def combinar_datos():
  train = pd.read_csv('./train.csv')
  test  = pd.read_csv('./test.csv')
  backup_targets = train.Survived
  train.drop(['Survived'],1,inplace = True)
  train_mas_test = train.append(test)
  train_mas_test.reset_index(inplace = True)
  train_mas_test.drop(['index','PassengerId'], inplace = True, axis = 1)
  return train_mas_test
train_mas_test = combinar_datos()

In [30]:
train_mas_test.describe()

,Pclass,Age,SibSp,Parch,Fare
count,1309.000000,1046.000000,1309.000000,1309.000000,1308.000000
mean,2.294882,29.881138,0.498854,0.385027,33.295479
std,0.837836,14.413493,1.041658,0.865560,51.758668
min,1.000000,0.170000,0.000000,0.000000,0.000000
25%,2.000000,21.000000,0.000000,0.000000,7.895800
50%,3.000000,28.000000,0.000000,0.000000,14.454200
75%,3.000000,39.000000,1.000000,0.000000,31.275000
max,3.000000,80.000000,8.000000,9.000000,512.329200


In [31]:
print(train_mas_test['Age'].median())
# 28
# Mas adelante la vamos a llenar con un promedio por grupos de personas
# data['Age'] = data['Age'].fillna(train_mas_test['Age'].median())

28.0


In [32]:
# la instruccion anterior llena los datos vacios de Age con el promedio de train mas test
data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [33]:
# Creamos una columna mas con el titulo de la persona
titulos = set()
for name in data['Name']:
  titulos.add(name.split(',')[1].split('.')[0].strip())
print(titulos)
# {'the Countess', 'Don', 'Master', 'Rev', 'Mme', 'Jonkheer', 'Major', 'Sir', 'Ms', 'Mr', 'Mlle', 'Dr', 'Col', 'Mrs', 'Miss', 'Lady', 'Capt'}

{'Miss', 'Col', 'Mrs', 'Mme', 'Major', 'Jonkheer', 'Sir', 'the Countess', 'Don', 'Rev', 'Lady', 'Mr', 'Mlle', 'Ms', 'Dr', 'Capt', 'Master'}


In [0]:
# haciendo un diccionario con los equivalentes
Diccionario_titulos = {
    "Capt": "Oficial",
    "Col": "Oficial",
    "Major": "Oficial",
    "Jonkheer": "Realeza",
    "Don": "Realeza",
    "Sir" : "Realeza",
    "Dr": "Oficial",
    "Rev": "Oficial",
    "the Countess":"Realeza",
    "Mme": "Mrs",
    "Mlle": "Miss",
    "Ms": "Mrs",
    "Mr" : "Mr",
    "Mrs" : "Mrs",
    "Miss" : "Miss",
    "Master" : "Master",
    "Lady" : "Realeza"
}

In [0]:
def obten_titulos():
  train_mas_test['Title'] = train_mas_test['Name'].map(lambda name:name.split(',')[1].split('.')[0].strip())
  train_mas_test['Title'] = train_mas_test.Title.map(Diccionario_titulos)
  return train_mas_test

In [36]:
train_mas_test = obten_titulos()
train_mas_test.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Mrs
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Mr


In [37]:
# verificar donde el titulo quedo en null, solo un caso pero no esta en el set de entrenamiento
train_mas_test[train_mas_test['Title'].isnull()]

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
1305,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9,C105,C,NaN


In [38]:
# tercera aproximacion a la edad, en lugar de llenarlo solo con el promedio de todo el data-set,
# se llenara de acuerdo a una agrupacion de otras columnas posterior a la agrupacion se aplicara el promedio
print(train_mas_test.iloc[:891].Age.isnull().sum())
#177
print(train_mas_test.iloc[891:].Age.isnull().sum())
#86


177
86


In [39]:
# examinando que tan diferentes pueden ser los valores de edad si se agrupan
grouped_train = train_mas_test.iloc[:891].groupby(['Sex','Pclass','Title'])
grouped_median_train = grouped_train.median()
grouped_median_train = grouped_median_train.reset_index()[['Sex', 'Pclass', 'Title', 'Age']]

grouped_median_train.head()

,Sex,Pclass,Title,Age
0,female,1,Miss,30.0
1,female,1,Mrs,40.0
2,female,1,Oficial,49.0
3,female,1,Realeza,40.5
4,female,2,Miss,24.0


In [40]:
grouped_train2 = train_mas_test.iloc[:891].groupby(['Sex','Pclass','Title','Embarked'])
grouped_median_train2 = grouped_train2.median()
grouped_median_train2 = grouped_median_train2.reset_index()[['Sex', 'Pclass', 'Title','Embarked', 'Age']]

grouped_median_train2.head()

,Sex,Pclass,Title,Embarked,Age
0,female,1,Miss,C,30.5
1,female,1,Miss,Q,33.0
2,female,1,Miss,S,23.0
3,female,1,Mrs,C,44.0
4,female,1,Mrs,S,39.0


In [0]:
def llena_edad(row):
    condition = (
        (grouped_median_train['Sex'] == row['Sex']) & 
        (grouped_median_train['Title'] == row['Title']) & 
        (grouped_median_train['Pclass'] == row['Pclass'])
        # & (grouped_median_train2['Embarked'] == row['Embarked']) 
    ) 
    return grouped_median_train[condition]['Age'].values[0]

def procesa_edad():
    global train_mas_test    
    train_mas_test['Age'] = train_mas_test.apply(lambda row: llena_edad(row) if np.isnan(row['Age']) else row['Age'], axis=1)
    return train_mas_test

train_mas_test = procesa_edad()

In [44]:
print(train_mas_test.shape)

(1309, 11)


In [0]:
def procesando_nombres():
    global train_mas_test
    # limpiamos la variable nombre
    train_mas_test.drop('Name', axis=1, inplace=True)
    
    # encoding in dummy variable
    titles_dummies = pd.get_dummies(train_mas_test['Title'], prefix='Title')
    train_mas_test = pd.concat([train_mas_test, titles_dummies], axis=1)
    
    # removing the title variable
    train_mas_test.drop('Title', axis=1, inplace=True)
        
    return train_mas_test

In [46]:
train_mas_test = procesando_nombres()

train_mas_test.head()

,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Oficial,Title_Realeza
0,3,male,22.0,1,0,A/5 21171,7.2500,NaN,S,0,0,1,0,0,0
1,1,female,38.0,1,0,PC 17599,71.2833,C85,C,0,0,0,1,0,0
2,3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0,1,0,0,0,0
3,1,female,35.0,1,0,113803,53.1000,C123,S,0,0,0,1,0,0
4,3,male,35.0,0,0,373450,8.0500,NaN,S,0,0,1,0,0,0


In [0]:
def procesando_fares():
    global train_mas_test
    # there's one missing fare value - replacing it with the mean.
    train_mas_test.Fare.fillna(train_mas_test.iloc[:891].Fare.mean(), inplace=True)
    return train_mas_test
train_mas_test = procesando_fares()

In [0]:
def procesando_embarked():
    global train_mas_test
    # two missing embarked values - filling them with the most frequent one in the train  set(S)
    train_mas_test.Embarked.fillna('S', inplace=True)
    # dummy encoding 
    embarked_dummies = pd.get_dummies(train_mas_test['Embarked'], prefix='Embarked')
    train_mas_test = pd.concat([train_mas_test, embarked_dummies], axis=1)
    train_mas_test.drop('Embarked', axis=1, inplace=True)
    return train_mas_test
train_mas_test = procesando_embarked()

In [51]:
train_mas_test.head()

,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Oficial,Title_Realeza,Embarked_C,Embarked_Q,Embarked_S
0,3,male,22.0,1,0,A/5 21171,7.2500,NaN,0,0,1,0,0,0,0,0,1
1,1,female,38.0,1,0,PC 17599,71.2833,C85,0,0,0,1,0,0,1,0,0
2,3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,0,1,0,0,0,0,0,0,1
3,1,female,35.0,1,0,113803,53.1000,C123,0,0,0,1,0,0,0,0,1
4,3,male,35.0,0,0,373450,8.0500,NaN,0,0,1,0,0,0,0,0,1


In [0]:
# No voy a procesar Cabin por que creo que es un dato irrelevante

In [0]:
def procesa_sexo():
    global train_mas_test
    # mapping string values to numerical one 
    train_mas_test['Sex'] = train_mas_test['Sex'].map({'male':1, 'female':0})
    return train_mas_test
train_mas_test = procesa_sexo()

In [0]:
def procesando_pclass():
    
    global train_mas_test
    # encoding into 3 categories:
    pclass_dummies = pd.get_dummies(train_mas_test['Pclass'], prefix="Pclass")
    
    # adding dummy variable
    train_mas_test = pd.concat([train_mas_test, pclass_dummies],axis=1)
    
    # removing "Pclass"
    train_mas_test.drop('Pclass',axis=1,inplace=True)
    
    return train_mas_test
train_mas_test = procesando_pclass()    

In [59]:
#Procesando el ticket
def cleanTicket(ticket):
    ticket = ticket.replace('.', '')
    ticket = ticket.replace('/', '')
    ticket = ticket.split()
    ticket = map(lambda t : t.strip(), ticket)
    ticket = list(filter(lambda t : not t.isdigit(), ticket))
    #print(ticket)
    #print(type(ticket))
    print(len(ticket))
    if len(ticket) > 0:
        return ticket[0]
    else: 
        return 'XXX'

tickets = set()
for t in train_mas_test['Ticket']:
    tickets.add(cleanTicket(t))

print(len(tickets))
#37


def procesando_ticket():
    
    global train_mas_test
    
    # a function that extracts each prefix of the ticket, returns 'XXX' if no prefix (i.e the ticket is a digit)
    def cleanTicket(ticket):
        ticket = ticket.replace('.','')
        ticket = ticket.replace('/','')
        ticket = ticket.split()
        ticket = map(lambda t : t.strip(), ticket)
        ticket = filter(lambda t : not t.isdigit(), ticket)
        if len(ticket) > 0:
            return ticket[0]
        else: 
            return 'XXX'
    

    # Extracting dummy variables from tickets:

    train_mas_test['Ticket'] = train_mas_test['Ticket'].map(cleanTicket)
    tickets_dummies = pd.get_dummies(train_mas_test['Ticket'], prefix='Ticket')
    train_mas_test = pd.concat([train_mas_test, tickets_dummies], axis=1)
    train_mas_test.drop('Ticket', inplace=True, axis=1)

    return train_mas_test

train_mas_test = procesando_ticket()

37


TypeError: ignored